# Weight and Cost Estimation
### EAE 130A

The following code outlines how we went about calculating an approximate weight for our concepts, as well as a cost estimate for the RDT&E of the aircraft. 

In [37]:
import math
import matplotlib as plt
import numpy as np

# WEIGHT ESTIMATION

For all of the concepts we developed, we used the same criterion for calculating the weight of the crew and the payload. Each crew is made up of 2 pilots and 1 attendant, each weighing approximately 190lbs and carrying 30lbs of baggage each. The payload includes 50 passengers, each weighing about 200lbs and carrying 40lbs of baggage each. These values came from the AIAA requirements for the aircraft design. Both the cruise speed, 350 knots, and the mission range of 1000 nmi were also taken from the AIAA requirements are the same for all the concepts. We chose a cruising altitude of 25000 ft as many similar aircraft such as the Dash 8 and the Dornier 328 cruise at that altitude [3]. Based on the Dash-8 we found a rate of climb of 1800 ft/min [8]. For propeller efficiency, a value between around 0.7 and 0.8 [2]. This value can be altered depending on the engine being used. The lift to drag ration (L/D) changes depending on the aircraft concept you are estimating the weight of. Comparable aircraft had values around 15 to 18 [4]. The endurace value is also subject to change depending on the mission profile. We settled on any value between 30 to 60 minutes for a given mission. For power specific fuel consumption, we found for the Dash 8 a PSFC of 0.483 and an efficiency of 0.311 [6]. We made an assumption that our aircraft would likely be fueled with A-1 jet fuel, which we found to have an energy density of 43.15 MJ/kg [1]. Additionally we predicted based on some projections that it would be possible for a battery density of 500 Wh/kg by 2035 [7].

# Weight estimation equations

For our weight estimation, we developed a "hybridization" factor that would allow us to control how much energy of each stage of the mission is used by the electric system. We called this term beta, and we have a unique beta value for each stage that we can control. The thought process was to calculate the amount of energy needed for a given stage. Then we apply our beta value to see how much of that energy needs to come from fuel and from the battery. Once we have those values, we can use specific energies of the fuel and batteries and energy density to calculate the mass of fuel or batteries needed to achieve the needed amount of energy. For this mission profile, we are assuming that beta will be 0.5 for takeoff and climb, and 0 for all other stages of the flight. Below are all the equations we used or developed to get our estimates:

Empty weight fraction estimation from Martins meta book:
$$ \frac{W_e}{W_o} = AW_o ^C $$

Ratio equation that describes the ratio of specific energy of fuel over specific energy of battery. We are assuming that the electric process has no inefficiencies, so we only multiply by the efficiency of the combustion engine process:

$$ ratio = \frac{e_f}{e_b}*eta_{comb} $$

This the equation that we are iterating to converge on the proper MTOW. W_f is the weight of the fuel and W_b is the weight of the battery:

$$ W_o = \frac{W_{crew}+W_{payload}}{(1-\frac{W_e}{W_o}-\frac{W_f}{W_o}-\frac{W_b}{W_o})} $$

Total fuel fraction for full mission profile:

$$ \frac{W_f}{W_o} = (1-\frac{W_1}{W_0}*\frac{W_2}{W_1}*\frac{W_3}{W_2}*\frac{W_4}{W_3}*\frac{W_5}{W_4}) * 1.06 $$

These are the equations derived for the individual fuel and battery fractions:

$$ \frac{W_{b,i}}{W_i} = (1-\frac{W_{i+1}}{W_{i}})[\frac{e^*_f}{e^*_b}\frac{\beta_i}{1-\beta_i}\frac{\eta_{comb}}{\eta_{pt}}] = \frac{W_{f,i}}{W_i}k_i $$

$$ \frac{W_b}{W_0} = k_0+\sum_{i=1}^{N-1}(k_i-k_{i-1})\frac{W_i}{W_0}-k_NW_N $$



In [38]:
# Number of pilots
pilots = 2
# Number of attendants
attend = 1
# Total crew
crew = pilots + attend
# Number of passengers
passengers = 50
# Weight of each crew member in lbs
wt_crew = 190
# Weight of each passenger in lbs
wt_passengers = 200
# Weight of baggage for each crew member in lbs
wt_crew_baggage = 30
# Weight of baggage for each passenger in lbs
wt_pass_baggage = 40
# Total payload weight
w_payload = (passengers * (wt_passengers + wt_pass_baggage))
# Total crew weight
w_crew = crew * (wt_crew + wt_crew_baggage)

# Mission range of 1000 nmi
R = 1000*6076.11549 # ft
# Cruise altitude 25000 ft
h = 25000 # ft
# rate of climb  1800 ft/min
ROC = 1800/60 # ft
# Propeller Efficiency of 0.8
eta_p = 0.8
# Force of gravity in ft/s^2
g = 32.17 
# Lift to drag ratio depending on aircraft design
L_D = 17 

# Target cruise speed in knots, assume target 350 ktas
V = 350*1.6878098571 # ft/s
# Assume an endurance of 45 min
E = 45*60 # s
# Assume PSFC of 0.483 lbm/(hp*hr)
PSFC_hp = 0.483
PSFC = (0.483) / (3600 * 550) # lbm/(fpf*s)
# Assume Jet A-1 fuel (43.15 MJ/kg)

# combustion efficiency
eta_comb = 0.311
e_f= 43.15*(1e6*1.3558179483314/2.20462262) # fpf/lbm
# Assume battery density to be (500 Wh/kg)
e_b= 500*(3600*1.3558179483314/2.20462262) # fpf/lbm
# Specific energy of fuel over specific energy of battery times combustion efficiency
ratio = e_f/e_b*(eta_comb)



def WeightEstimation(B,W0_guess):
    # format for hybrid parameter:
    # B = [Taxi&Takeoff, Climb, Cruise, Descent, Loiter, Landing]


    # Mission Weight fractions
    W1_W0 = 1 - 0.0004*PSFC_hp*(1-B[0])
    W2_W1 = 1 - h*PSFC*(1-B[1])*0.15*550/ROC
    W3_W2 = np.exp((-R*PSFC*(1-B[2]) )/ (eta_p*L_D))
    W4_W3 = 1 - h*PSFC*(1-B[3])*0.15*550/ROC
    W5_W4 = np.exp((-E*PSFC*V*(1-B[4])) / (eta_p*L_D))
    Wb_W0 = (B[0]/(1-B[0]) + (B[1]/(1-B[1])-B[0]/(1-B[0]))*W1_W0 + (B[2]/(1-B[2])-B[1]/(1-B[1]))*W1_W0*W2_W1 + (B[3]/(1-B[3])-B[2]/(1-B[2]))*W1_W0*W2_W1*W3_W2 + (B[4]/(1-B[4])-B[3]/(1-B[3]))*W1_W0*W2_W1*W3_W2*W4_W3 - B[4]/(1-B[4])*W1_W0*W2_W1*W3_W2*W4_W3*W5_W4)*ratio
    Wf_W0 = (1 - W1_W0*W2_W1*W3_W2*W4_W3*W5_W4) * 1.06

    # Calculate Weight
    W0_list = []
    W0 = W0_guess
    A = 0.96
    C = -0.05
    error = 1e-6
    delta  = (2 * error)

    while delta > error:
        W0_list.append(W0)
        We_W0 = A*W0**C
        w0_New = (w_crew + w_payload) / (1 - We_W0 - Wf_W0 - Wb_W0)
        delta = abs((w0_New - W0)/w0_New)
        W0 = w0_New

    return W0,We_W0,Wf_W0,Wb_W0

# format for hybrid parameter:
# B = [Taxi&Takeoff, Climb, Cruise, Descent, Loiter, Landing]
B=[0.5,0.5,0,0,0,0]
W0_guess = 28000
W0,We_W0,Wf_W0,Wb_W0 = WeightEstimation(B,W0_guess)

# m_batt = (R * W0) / (eta_p * e * L_D)
We = We_W0 * W0
# # W_elec = W0 - w_crew - w_payload - We
W_elec = W0*Wb_W0
print("Fuel Fraction: {:3f}".format(Wf_W0))
print("Fuel Weight: {:3f}".format(Wf_W0*W0))
print("Battery Fraction: {:3f}".format(Wb_W0))
print("Battery Weight: {:3f}".format(Wb_W0*W0))
print("Takeoff Gross Weight: " + str(round(W0)) + " lbs")
print("Empty Weight: " + str(round(We)) + " lbs")
print("Empty Weight Fraction: {:.3f}".format(We_W0))
print("Battery & Motor Weight: " + str(round(W_elec)) + " lbs")
print("Battery & Motor Weight Fraction: {:.3f}".format(We_W0))
print("Weight of Crew and Payload {:.3f} lbs".format(w_crew+w_payload))

Fuel Fraction: 0.159456
Fuel Weight: 9089.957979
Battery Fraction: 0.063230
Battery Weight: 3604.527924
Takeoff Gross Weight: 57006 lbs
Empty Weight: 31652 lbs
Empty Weight Fraction: 0.555
Battery & Motor Weight: 3605 lbs
Battery & Motor Weight Fraction: 0.555
Weight of Crew and Payload 12660.000 lbs


# COST ESTIMATION

For finding the approximate cost of our concepts, we started with a set of criterion that we applied to all of them. We chose a cruise velocity of 350 knots as per the AIAA RFP. We decided that it was reasonable for our company to make 100 aircraft in 5 years with 5 prototypes for testing, which equates to about 1.75 aircraft per month. The number of propellers, electric motors, and combustion engines all depended on the concept we were working with. We used the given CPI of 1.31 from the lecture slides in class. We decided for our cost estimation to use the Dash-8 engine as a basis for the engine specs we needed to enter. We found a shaft horse power of about 2500hp, which we split evenly between the electric and combustion motors [6]. Additionally, we decided to make a guess that by 2035 it would be possible for our plane to have a battery capacity of 1500 kWh. Basing our propeller off the Dash 8, it has a propeller diameter of 13.45 ft [5].

# Cost estimate equations


Engineering Costs
$$
C_{\mathrm{ENG}}=0.083 \cdot W_{\text {Airframe }}^{0.791} \cdot V_H^{1.521} \cdot Q^{0.183} \cdot F_{\text {Cert }} \cdot F_{C F} \cdot F_{\text {Comp }} \cdot F_{\text {Press }} \cdot F_{\mathrm{HyE}} \cdot R_{\mathrm{ENG}} \cdot C P I
$$
Tooling costs
$$
C_{\mathrm{TOOL}}=2.1036 \cdot W_{\mathrm{Airframe}}^{0.764} \cdot V_H^{0.899} \cdot Q^{0.178} \cdot Q_M^{0.066} \cdot F_{\mathrm{Taper}} \cdot F_{C F} \cdot F_{\text {Comp }} \cdot F_{\mathrm{Press}} \cdot F_{\mathrm{HyE}} \cdot R_{\mathrm{TOOL}} \cdot C P I
$$
Manufacturing costs
$$
C_{\mathrm{MFG}}=20.2588 \cdot W_{\text {Airframe }}^{0.74} \cdot V_H^{0.543} \cdot Q^{0.524} \cdot F_{\text {Cert }} \cdot F_{C F} \cdot F_{\text {Comp }} \cdot F_{H y E} \cdot R_{M F G} \cdot C P I
$$
Development support costs
$$
C_{\mathrm{DEV}}=0.06458 \cdot W_{\text {Airframe }}^{0.873} \cdot V_H^{1.89} \cdot Q_{\text {Proto }}^{0.346} \cdot F_{\text {Cert }} \cdot F_{C F} \cdot F_{\text {Comp }} \cdot F_{\text {Press }} \cdot F_{\mathrm{HyE}} \cdot C P I
$$
Flight test operations costs 
$$
C_{Q C}=0.13 \cdot C_{\mathrm{MFG}} \cdot F_{\mathrm{CERT}} \cdot F_{\mathrm{COMP}} \cdot F_{\mathrm{HyE}}
$$
Electric motor cost
$$
C_{\mathrm{EM}}=174 \cdot N_{\mathrm{Motor}} \cdot P_{\mathrm{EM}} \cdot C P I
$$
Power management system cost
$$
C_{\text {PMS }}=150 \cdot P_{\text {EM,total }} \cdot C P I
$$
Battery cost (assumes $200 \$ / \mathrm{kWh}$ )
$$
C_{B A T}=200 \cdot E_{B A T} \cdot C P I
$$
Propeller costs
$$
C_{\text {Prop }}=210 \cdot N_{\text {Prop }} \cdot C P I \cdot D_P^2 \cdot\left(\frac{P_{S H P}}{D_P}\right)^{0.12}
$$


$$
\begin{array}{|l|c|l|l|l|l|l|}
\hline \text { CER Category } & \begin{array}{l}
\text { FCert } \\
\text { Factor for } \\
\text { LSA } \\
\text { certification }
\end{array} & \begin{array}{l}
\text { F Comp } \\
\text { Factor for } \\
100 \% \\
\text { composites }
\end{array} & \begin{array}{l}
\text { F Faper } \\
\text { Factor for } \\
\text { untapered } \\
\text { wings }
\end{array} & \begin{array}{l}
\text { FF } \\
\text { Factor for } \\
\text { complex } \\
\text { flap system }
\end{array} & \begin{array}{l}
\text { F Factor for } \\
\text { pressurized } \\
\text { cabin }
\end{array} & \begin{array}{l}
\text { F } \\
\text { Factor for } \\
\text { hybrid-electric } \\
\text { propulsion }
\end{array} \\
\hline \begin{array}{l}
\text { Engineering } \\
\text { Cost }
\end{array} & 0.67 & 2.00 & - & 1.03 & 1.03 & 1.33-1.66 \\
\hline \begin{array}{l}
\text { Tooling } \\
\text { Cost }
\end{array} & - & 2.00 & 0.95 & 1.02 & 1.01 & 1.10 \\
\hline \begin{array}{l}
\text { Manufacturing } \\
\text { Cost }
\end{array} & 0.75 & 1.25 & - & 1.01 & - & 1.10 \\
\hline \begin{array}{l}
\text { Development } \\
\text { Support Cost }
\end{array} & 0.50 & 1.50 & - & 1.01 & 1.03 & 1.05 \\
\hline \begin{array}{l}
\text { Flight Test } \\
\text { Operations Cost }
\end{array} & 0.50 & - & - & - & - & 1.50 \\
\hline \begin{array}{l}
\text { Quality Control } \\
\text { Cost }
\end{array} & 0.50 & 1.50 & - & - & - & 1.50 \\
\hline \begin{array}{l}
\text { Materials } \\
\text { Cost }
\end{array} & 0.75 & - & - & 1.02 & 1.01 & 1.05 \\
\hline
\end{array} \\
$$

Airframe weight estimation:

$$ W_{airframe} = 0.22 * W_e $$

Hourly rate estimations for tooling, engineering, labor, etc. with y as the year:

$$ R_{tool} = 2.883y - 5666 $$
$$ R_{eng} = 2.575y - 5058 $$
$$ R_{mfg} = 2.316y - 4552 $$
$$ R_{qc} = 2.60y - 5112 $$ 

In [39]:
# Define variables
# W_airframe = We*.65       #  weight (lbs)
V_max = 350        # Maximum level airspeed (KTAS - knots true airspeed)
Q = 100            # number of aircraft to be produced over 5 year period
Q_proto = 5      # number of prototypes to be produced
Q_m = 1.75          # number of aircraft made per month
N_prop = 2          # number of propellers
cpi = 1.31          # consumer price index for consideration of inflation
year = 2023         # current year (used for R calculations)
N_engine = 2     # number of combustion engines
N_motor = 2      # number of electric motors
P_em = 932.12         # Power of electric motor in kW
P_shp = 2500        # Shaft horse power
P_ice = 1250        # power of combustion engine in hp
D_p = 13.45          # diameter of propeller in feet
E_bat = 1500      # battery capacity in kWh


# Define factors
# Indexed based on type of cost
# Engineering 0, Tooling 1, Mfg 2, Dev 3, Flight test 4, Quality control 5, Materials 6     
F_cert =  [0.67, 1.00, 0.75, 0.50, 0.50, 0.50, 0.75]
F_comp =  [2.00, 2.00, 1.25, 1.50, 1.00, 1.50, 1.00]
F_taper = [1.00, 0.95, 1.00, 1.00, 1.00, 1.00, 1.00]
F_cf =    [1.03, 1.02, 1.01, 1.01, 1.00, 1.00, 1.02]
F_press = [1.03, 1.01, 1.00, 1.03, 1.00, 1.00, 1.01]
F_hye =   [1.33, 1.10, 1.10, 1.05, 1.50, 1.50, 1.05]

W_airframe =  We*.22 
R_tool = 2.883 * year - 5666
R_eng = 2.575 * year - 5058
R_mfg = 2.316 * year - 4552
R_qc = 2.60 * year - 5112

P_emtotal = N_motor * P_em

In [40]:
# RDT&E and Production CERs

# Engineering costs
C_eng = 0.083 * W_airframe**0.791 * V_max**1.521 * Q**0.183 * F_cert[0] * F_cf[0] * F_comp[0] * F_press[0] * F_hye[0] * R_eng * cpi
print('Engineering: $', C_eng)

# Tooling costs
C_tool = 2.1036 * W_airframe**0.764 * V_max**0.899 * Q**0.178 * Q_m**0.066 * F_taper[1] * F_cf[1] * F_comp[1] * F_press[1] * F_hye[1] * R_tool * cpi 
print('Tooling: $', C_tool)

# Manufacturing costs
C_mfg = 20.2588 * W_airframe**0.74 * V_max**0.543 * Q**0.524 * F_cert[2] * F_cf[2] * F_comp[2] * F_hye[2] * R_mfg * cpi 
print('Manufacturing: $', C_mfg)

# Development support costs
C_dev = 0.06458 * W_airframe**0.873 * V_max**1.89 * Q_proto**0.346 * F_cert[3] * F_cf[3], F_comp[3] * F_press[3] * F_hye[3] * cpi 
C_dev = C_dev[0]
print('Development Support: $', C_dev)

# Flight test operations costs
C_op = 0.009646 * W_airframe**1.16 * V_max**1.3718 * Q_proto**1.281 * F_cert[4] * F_hye[4] * cpi 
print('Flight test operations: $', C_op)

# Quality Control costs
C_qc = 0.13 * C_mfg * F_cert[5] * F_comp[5] * F_hye[5]
print('Quality Control: $', C_qc)

# Materials costs
C_mat = 24.896 * W_airframe**0.689 * V_max**0.624 * Q**0.792 * cpi * F_cert[6] * F_cf[6] * F_press[6] * F_hye[6]
print('Materials: $', C_mat)

# Combusion engine costs
C_ice = 174 * N_engine * P_ice * cpi
print('Combusion Engine: $', C_eng)

# Electric motor cost
C_em = 174 * N_motor * P_em * cpi
print('Electric motor: $', C_em)

# Power management system cost
C_pms = 150 * P_emtotal * cpi 
print('Power Management System: $', C_pms)

# Battery cost (assumes $200 / kWh)
C_bat = 200 * E_bat * cpi 
print('Battery: $', C_bat)

# Propeller costs
C_prop = 210 * N_prop * cpi * D_p**2 * (P_shp / D_p)**0.12
print('Propeller: $', C_prop)


#### Total RDT&E production cost
C_rdte = C_eng + C_tool + C_mfg + C_dev + C_op + C_qc + C_mat + C_ice + C_em + C_pms + C_bat + C_prop
print('Total: $', C_rdte)

C_rdte_per_plane = C_rdte / (Q)
print('Cost per plane: $', C_rdte_per_plane)

Engineering: $ 585870837.1809834
Tooling: $ 388409732.4973183
Manufacturing: $ 690894727.305827
Development Support: $ 8285207.4798663175
Flight test operations: $ 6602128.636398139
Quality Control: $ 101043353.86847721
Materials: $ 17449152.06614477
Combusion Engine: $ 585870837.1809834
Electric motor: $ 424934.8656
Power Management System: $ 366323.16000000003
Battery: $ 393000.0
Propeller: $ 186325.06510444242
Total: $ 1800495572.1257198
Cost per plane: $ 18004955.7212572


# Sources

[1]     Greenbaum, W., Alternative Jet Fuels, Dec. 2012, http://large.stanford.edu/courses/2012/ph240/greenbaum1/#:~:text=Jet%20A%2D1%20fuel%20has,be%20achieved%20by%20liquid%20fuels

[2]     Spakovsky, Z. S., 11.7 Performance of Propellers, April 2007, https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html#:~:text=In%20practice%2C%20the%20propulsive%20efficiency,shown%20in%20the%20following%20section 

[3]     Palt, K., “Dornier / Fairchild-Dornier Do 328 / 328-100 Regional Airliner,” Dornier / Fairchild-Dornier do 328 / 328-100 - specifications - technical data / description Available: http://www.flugzeuginfo.net/acdata_php/acdata_do328_en.php. 

[4]     Filippone, A., “Aerodynamic Database Lift-to-Drag Ratios,” Advanced Topics in Aerodynamics, Oct. 2004, https://aerodyn.org/ld-tables/

[5]     Aughinbaugh II , D., Bombardier Q400 Propeller - Dowty R408, Jun. 2015, https://www.flyradius.com/bombardier-q400/propeller-dowty-r408 

[6]     Pratt & Whitney Canada PW100, retrieved 1 February 2023
https://engineering.purdue.edu/~propulsi/propulsion/jets/tprops/pw100.html. 

[7]     Bills, A. et al., “Performance metrics required of next-generation ... - ACS publications,” Performance Metrics Required of Next-Generation Batteries to Electrify Commercial Aircraft, Feb. 2020, https://pubs.acs.org/doi/10.1021/acsenergylett.9b02574. 

[8]     Frawley, G., “De Havilland Canada DHC-8-300 Dash 8,” Airliners.net, Feb. 2017, https://www.airliners.net/aircraft-data/de-havilland-canada-dhc-8-300-dash-8/121  